In [2]:
import pandas as pd
import numpy as np
import random

In [18]:
#Скачать данные с сайта

from app import ReestrRequest
from src.queries import lfilt
data = ReestrRequest(lfilt['oil']).get_data_from_reestr()
df = pd.DataFrame(data)

In [14]:
#Загрузить из файла
from analysis.import_path import imp_path
df = pd.read_excel(imp_path())

    PIVOT_CREATING

Подготовка данных

In [ ]:
#Создание датафрейма с номерами послединих лицензий
df_last = df[['Наименование участка недр','Государственный регистрационный номер', 'Дата лицензии', 'Year']][df.Last == True]
df_last['prev_lic'] = df_last['Государственный регистрационный номер']

#Создание датафрейма для присоединения к мейну
df_last_pivoted = df_last.pivot(index=['Наименование участка недр','Государственный регистрационный номер', 'Дата лицензии'], columns=['Year'])['prev_lic'].sort_values(by = ['Year'], axis=1, ascending=False).bfill(axis = 1).ffill(axis = 1)

In [ ]:
#Создание основного датафрейма для лицензий
df_main = df[['Наименование участка недр','Государственный регистрационный номер','Дата лицензии', 'Year', 'prev_lic']].sort_values('Year', ascending=False)

#добавление "_" к значению
df_main['prev_lic'] = df_main['prev_lic'].apply(lambda x: '_' + x if type(x) is str else np.nan)

#Создание пивот-датафрейма с столбцами в убывающем порядке 
df_pivoted = df_main.pivot(index=['Наименование участка недр','Государственный регистрационный номер', 'Дата лицензии'], columns=['Year'])['prev_lic'].sort_values(by = ['Year'], axis=1, ascending=False).ffill(axis=1)

In [ ]:
df_concat = pd.concat([df_pivoted, df_last_pivoted]).sort_index(level=[0,2])

In [ ]:
df_nan = df_concat[(df_concat.index.get_level_values(0)).isna()]

In [ ]:
df_concat

----

In [ ]:
#Взять случайный номер лизензии без наименования участка недр
rnd = random.choice(df['Государственный регистрационный номер'][df['Наименование участка недр'].isna()].tolist())
print (str(rnd))

df_pivoted[(df_pivoted.index.get_level_values(0).isna()) & df_pivoted.index.get_level_values(1).isin([rnd])]

In [ ]:
df[df.index.isin(['ВЛГ00292НЭ','ВЛГ01572НЭ','ВЛГ00572НЭ','ВЛГ01723НЭ'])]